In [1]:
print('hello')

hello


In [1]:
import cv2
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input
import os

2024-10-16 16:38:18.258103: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 16:38:18.269322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 16:38:18.280827: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 16:38:18.284279: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 16:38:18.294547: I tensorflow/core/platform/cpu_feature_guar

In [4]:
from ultralytics import YOLO
from tensorflow import keras

# Load YOLOv9c model (PyTorch)
yolo_model = YOLO('/home/research/Kilsar_Sentinal/Mohammad/runs/detect/train47/weights/best.pt')
# yolo_model = torch.hub.load('ultralytics/yolov9c', 'custom', path='/home/research/Kilsar_Sentinal/Mohammad/runs/detect/train47/weights/best.pt')

# Load EfficientNet model (Keras)
effnet_model = keras.models.load_model('/home/research/Kilsar_Sentinal/suryansh/src/models/tf models/model-1.keras')


2024-10-16 16:39:06.822390: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22400 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:4a:00.0, compute capability: 8.0
2024-10-16 16:39:06.824441: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22400 MB memory:  -> device: 1, name: NVIDIA A30, pci bus id: 0000:61:00.0, compute capability: 8.0
2024-10-16 16:39:06.826394: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 22400 MB memory:  -> device: 2, name: NVIDIA A30, pci bus id: 0000:ca:00.0, compute capability: 8.0
2024-10-16 16:39:06.828605: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 22400 MB memory:  -> device: 3, name: NVIDIA A30, pci bus id: 0000:e1:00.0, compute capability: 8.0


In [2]:
yolo_model.info()
effnet_model.summary()

YOLOv9c summary: 618 layers, 25,530,003 parameters, 0 gradients, 103.7 GFLOPs


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ (None, 1536)           │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       393,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,636,789 (44.39 MB)

 Trainable params: 426,626 (1.63 MB)

 Non-trainable params: 10,783,535 (41.14 MB)

 Optimizer params: 426,628 (1.63 MB)

In [22]:
import cv2
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input
import os

# Preprocess image for EfficientNet
def preprocess_for_effnet(crop):
    crop_resized = cv2.resize(crop, (300, 300))  # Resize to 300x300 for EfficientNet B3
    crop_preprocessed = preprocess_input(crop_resized)  # Apply EfficientNet preprocessing
    crop_preprocessed = np.expand_dims(crop_preprocessed, axis=0)  # Add batch dimension
    return crop_preprocessed

# Detection and classification pipeline for a single image
def detect_and_classify(image_path, output_folder):
    # Load the image
    image = cv2.imread(image_path)

    # Detect guns using YOLOv9c
    results = yolo_model(image)
    print(results[0].boxes)
    # gun_boxes = results.xyxy[0]  # Extract bounding boxes for detected guns
    gun_boxes = results.pred[0].cpu().numpy()

    # For each detected gun, classify it using EfficientNet
    for box in gun_boxes:
        x1, y1, x2, y2, conf, cls = map(int, box[:6])
        gun_crop = image[y1:y2, x1:x2]  # Crop the gun region

        # Preprocess for EfficientNet
        gun_crop_preprocessed = preprocess_for_effnet(gun_crop)

        # Classify the gun with EfficientNet
        predictions = effnet_model.predict(gun_crop_preprocessed)
        predicted_class = np.argmax(predictions, axis=1)
        class_label = 'Class 1' if predicted_class == 0 else 'Class 2'

        # Annotate the image with classification result
        cv2.putText(image, f'{class_label}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # Save the result to the specified output folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_filename = os.path.join(output_folder, f"output_{os.path.basename(image_path)}")
    cv2.imwrite(output_filename, image)

    return output_filename

Working code

In [25]:
import cv2
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input
import os

# Preprocess image for EfficientNet
def preprocess_for_effnet(crop):
    crop_resized = cv2.resize(crop, (300, 300))  # Resize to 300x300 for EfficientNet B3
    crop_preprocessed = preprocess_input(crop_resized)  # Apply EfficientNet preprocessing
    crop_preprocessed = np.expand_dims(crop_preprocessed, axis=0)  # Add batch dimension
    return crop_preprocessed

# Detection and classification pipeline for a single image
def detect_and_classify(image_path, output_folder):
    # Load the image
    image = cv2.imread(image_path)

    # Detect guns using YOLOv9c
    results = yolo_model(image)

    # Extract bounding boxes for detected guns
    gun_boxes = results[0].boxes

    # Initialize counts for each class
    class_counts = {'Class 1': 0, 'Class 2': 0}

    # For each detected gun, classify it using EfficientNet
    if gun_boxes is not None and len(gun_boxes) > 0:
        for box in gun_boxes:
            # Get bounding box coordinates, confidence, and class
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())

            # Use names dictionary for class label (if applicable)
            class_label = results[0].names[cls]

            # Crop the gun region from the image
            gun_crop = image[y1:y2, x1:x2]

            # Preprocess for EfficientNet
            gun_crop_preprocessed = preprocess_for_effnet(gun_crop)

            # Classify the gun with EfficientNet
            predictions = effnet_model.predict(gun_crop_preprocessed)
            predicted_class = np.argmax(predictions, axis=1)

            # Update counts based on the predicted class
            if predicted_class == 0:
                class_counts['Class 1'] += 1
            else:
                class_counts['Class 2'] += 1

            # Annotate the image with classification result
            cv2.putText(image, f'{class_label} ({conf:.2f})', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # Save the result to the specified output folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_filename = os.path.join(output_folder, f"output_{os.path.basename(image_path)}")
    cv2.imwrite(output_filename, image)

    # Print the counts of each class
    # print(f"Total Class 1: {class_counts['Class 1']}")
    # print(f"Total Class 2: {class_counts['Class 2']}")

    return output_filename


working code correct with results iteration

In [6]:
# Preprocess image for EfficientNet
def preprocess_for_effnet(crop):
    crop_resized = cv2.resize(crop, (300, 300))  # Resize to 300x300 for EfficientNet B3
    crop_preprocessed = preprocess_input(crop_resized)  # Apply EfficientNet preprocessing
    crop_preprocessed = np.expand_dims(crop_preprocessed, axis=0)  # Add batch dimension
    return crop_preprocessed


# Detection and classification pipeline for a single image
def detect_and_classify(image_path, output_folder):
    # Load the image
    image = cv2.imread(image_path)

    # Detect guns using YOLOv9c
    results = yolo_model(image)

    # Initialize counts for each class
    class_counts = {'Class 1': 0, 'Class 2': 0}
    class_predictions = []  # List to store predictions for this image

    # Iterate over the list of results
    for result in results:
        # Extract bounding boxes for detected guns
        gun_boxes = result.boxes

        # For each detected gun, classify it using EfficientNet
        if gun_boxes is not None and len(gun_boxes) > 0:
            for box in gun_boxes:
                # Get bounding box coordinates, confidence, and class
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())

                # Use names dictionary for class label (if applicable)
                class_label = result.names[cls]

                # Crop the gun region from the image
                gun_crop = image[y1:y2, x1:x2]

                # Preprocess for EfficientNet
                gun_crop_preprocessed = preprocess_for_effnet(gun_crop)

                # Classify the gun with EfficientNet
                predictions = effnet_model.predict(gun_crop_preprocessed)
                predicted_class = np.argmax(predictions, axis=1)

                # Check confidence for the predicted class
                class_confidence = predictions[0][predicted_class]  # Get confidence of the predicted class

                # Update counts based on the predicted class
                if class_confidence < 0.5:
                    # Print statement if confidence is below threshold
                    print(f"Low confidence for {class_label} : {class_confidence[0]:.2f}. Skipping bounding box.")
                    continue  # Skip this box and do not draw the bounding box

                if predicted_class == 0:
                    class_counts['Class 1'] += 1
                    class_predictions.append('Class 1')
                else:
                    class_counts['Class 2'] += 1
                    class_predictions.append('Class 2')

                # Annotate the image with classification result
                cv2.putText(image, f'{class_predictions[-1]} ({conf:.2f})', (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # Save the result to the specified output folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_filename = os.path.join(output_folder, f"output_{os.path.basename(image_path)}")
    cv2.imwrite(output_filename, image)

    # Print the counts of each class
    print(f"Total Class 1: {class_counts['Class 1']}")
    print(f"Total Class 2: {class_counts['Class 2']}")

    # Log results for this image
    print(f"Predictions for {os.path.basename(image_path)}: {class_predictions}")

    return output_filename

In [8]:


# Folder paths
input_folder = '/home/research/Kilsar_Sentinal/Data_results/demo'
output_folder = '/home/research/Kilsar_Sentinal/Data_results/cycle-6'

# Process all images in the input folder
for image_file in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_file)
    if image_file.endswith(('.png', '.jpg', '.jpeg')):
        output_image_path = detect_and_classify(image_path, output_folder)
        print(f"Processed {image_path}, saved result to {output_image_path}")


0: 448x640 (no detections), 11.6ms
Speed: 3.7ms preprocess, 11.6ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)
Total Class 1: 0
Total Class 2: 0
Predictions for istockphoto-1041100896-612x612.jpg: []
Processed /home/research/Kilsar_Sentinal/Data_results/demo/istockphoto-1041100896-612x612.jpg, saved result to /home/research/Kilsar_Sentinal/Data_results/cycle-6/output_istockphoto-1041100896-612x612.jpg


In [8]:
# import cv2

# cap = cv2.VideoCapture(0)  # Capture from webcam

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break
    
#     # Process the frame for detection and classification
#     ...
    
#     # Display the frame
#     cv2.imshow('Gun Detection and Classification', frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()